## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-31-18-54-13 +0000,nypost,"Rider goes through McDonald’s on horseback, tr...",https://nypost.com/2025/12/31/us-news/rider-go...
1,2025-12-31-18-54-00 +0000,wsj,Comex Gold Ends the Year 64.52% Higher at $432...,https://www.wsj.com/finance/commodities-future...
2,2025-12-31-18-46-24 +0000,nyt,"Ben Nighthorse Campbell, Former Colorado Senat...",https://www.nytimes.com/2025/12/30/us/politics...
3,2025-12-31-18-46-02 +0000,nyt,New Year’s 2026 Celebrations Around the World:...,https://www.nytimes.com/2025/12/31/world/asia/...
4,2025-12-31-18-41-15 +0000,bbc,"Fireworks, drums and light shows: How the worl...",https://www.bbc.com/news/videos/cd6wyll5338o?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2413/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,year,31
23,new,22
109,trump,20
82,ukraine,9
533,minnesota,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...,93
106,2025-12-31-11-05-20 +0000,nypost,Washington Monument will glow on New Year’s Ev...,https://nypost.com/2025/12/31/us-news/washingt...,83
12,2025-12-31-18-18-12 +0000,nypost,NYC New Year’s Eve revelers could face coldest...,https://nypost.com/2025/12/31/us-news/nyc-reve...,80
11,2025-12-31-18-19-37 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,80
42,2025-12-31-16-50-19 +0000,nypost,NYC among Northeast cities that could see snow...,https://nypost.com/2025/12/31/us-news/nyc-amon...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,93,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...
177,70,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...
17,37,2025-12-31-17-56-58 +0000,nypost,OpenAI’s pay tops every major tech startup as ...,https://nypost.com/2025/12/31/business/openais...
50,36,2025-12-31-16-18-53 +0000,nypost,Zohran Mamdani will become first NYC mayor swo...,https://nypost.com/2025/12/31/us-news/zohran-m...
72,31,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
153,31,2025-12-31-05-20-55 +0000,nyt,"Tatiana Schlossberg, Kennedy Daughter Who Wrot...",https://www.nytimes.com/2025/12/30/us/politics...
122,30,2025-12-31-10-30-00 +0000,wsj,Instead of crumbling under the highest U.S. ta...,https://www.wsj.com/economy/trade/trump-trade-...
15,28,2025-12-31-18-01-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
24,28,2025-12-31-17-38-52 +0000,latimes,Disney settles with DOJ for alleged child priv...,https://www.latimes.com/entertainment-arts/bus...
104,27,2025-12-31-11-25-40 +0000,cbc,U.S. lawyers to review millions of pages of Ep...,https://www.cbc.ca/news/world/epstein-files-re...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
